In [ ]:
from STK_Sim import *

Filename = 'AERO_401_Initial_Assessment'

stk_object = STK_Simulation(False,Filename)

In [ ]:
Target_Load_Option=0
Num_Targets=15
stk_object.Target_Loader(Target_Load_Option,Num_Targets)

In [ ]:
Satellite_Load_Option=0
Num_Satellites=12
per_range=(100,1000)
ago_range=(100,1000)
inc_range=(0,90)
aop_range=(45,46)
asc_range=(45,46)
loc_range=(0,90)
stk_object.Satellite_Loader(Satellite_Load_Option,Num_Satellites,per_range,ago_range,inc_range,aop_range,asc_range,loc_range)

In [ ]:
stk_object.plot_FOM(AgEFmDefinitionType.eFmNumberOfAccesses)